In [ ]:
%pip install --upgrade langchain-community langchain-core langchain-google-vertexai anthropic[vertex]

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Gemini Pro Prompt

In [ ]:
from langchain_google_vertexai import VertexAI

# To use model
model = VertexAI(model_name="gemini-pro")

message = "Tell me a joke about programming in LISP"
response = model.invoke(message)

print(response)

## Streaming

In [ ]:
import sys

for chunk in model.stream("Tell me a scary poem about Java Programming"):
    print(chunk, end="", flush=True)

## Anthropic Claude

In [ ]:
from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    HumanMessage,
    SystemMessage,
)
from langchain_core.outputs import LLMResult
from langchain_google_vertexai.model_garden import ChatAnthropicVertex

In [ ]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

# Initialise the Model
model = ChatAnthropicVertex(
    model_name="claude-3-sonnet",
    project=project,
    location=location,
)

In [ ]:
# prepare input data for the model
raw_context = (
    "Your name is Jerry, you are a Standup commedian"
    "Answer the question, and also tell a joke"
)
question = (
    "What is a good movie for kids?"
)
context = SystemMessage(content=raw_context)
message = HumanMessage(content=question)

# Invoke the model
response = model.invoke([context, message])
print(response.content)

## Simple Chat App

In [ ]:
# Array to store the message history
messages = []

# Chatbot context
raw_context = (
    """
    Your name is Joey, you are an expert in Python coding and SQL.
    You are a very nice and helpful.
    """
)

# Add the context to the Messages array
messages.append(SystemMessage(content=raw_context))

# Function to get a response (can be replaced with any logic or model)
def get_response(user_input):

    # Add the users question to the messages array
    messages.append(HumanMessage(content=user_input))

    # Invoke the model
    response = model.invoke(messages)

    # All the model response to the messages array
    messages.append(AIMessage(content=response.content))

    # Return the response
    return response.content

# Loop to keep the chat running
while True:
    # Get input from the user
    user_input = input("You: ")

    # Check if the user wants to exit the chat
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("ChatBot: Goodbye!")
        break

    # Get the response
    response = get_response(user_input)

    # Display the response
    print(f"ChatBot: {response}")


In [ ]:
# See the chat history
for message in messages:
    print(message)

# reset the chat
messages = []

## BigQuery Loader

In [ ]:
%pip install --upgrade --quiet langchain-google-community[bigquery]

In [ ]:
# The query returns an article from Hacker News
query = """
SELECT text
FROM `bigquery-public-data.hacker_news.full`
where title = "Another AirBnB Host Horror Story"
limit 1;
"""

In [ ]:
from langchain_google_community import BigQueryLoader

loader = BigQueryLoader(query)

data = loader.load()

print(data)
print("-" * 80)
print(data[0].page_content)

In [ ]:
model = VertexAI(model_name="gemini-pro")

prompt = """
Summarize the following article in one sentance,plus a few short bullets

Article: {0}

Summary:
""".format(data[0].page_content)

response = model.invoke(prompt)

print(response)


## Document AI

In [ ]:
%pip install --upgrade --quiet  langchain-google-community[docai]

In [ ]:
# Need to create a Cloud Storage Bucket in your project and specify here:
GCS_OUTPUT_PATH = "gs://ocr-processor-output-dar"

# Need to create a Document AI OCR Processor in your project
# Create in the Console
# Specify the ID here
PROCESSOR_NAME = "projects/982785856251/locations/us/processors/fef610830bfdc9f2"

In [ ]:
from langchain_core.document_loaders.blob_loaders import Blob
from langchain_google_community import DocAIParser

parser = DocAIParser(
    location="us", processor_name=PROCESSOR_NAME, gcs_output_path=GCS_OUTPUT_PATH
)

blob = Blob(
    path="gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/2022Q1_alphabet_earnings_release.pdf"
)

docs = list(parser.lazy_parse(blob))
print(len(docs))


In [ ]:
# print the first 200 characters of each doc
for doc in docs:
    print(doc.page_content[:200])
    print("-" * 80)

In [ ]:
# Concatenate all the pages together
DOC = " ".join([doc.page_content for doc in docs])
print(len(DOC))


In [ ]:
# Use Gemini 1.5 which spports up to 2M token conext (much larger than this example)
model = VertexAI(model_name="gemini-1.5-pro-001")

prompt = """
Summarize the following document in a couple sentances,
plus a few short bullets that highlight the most important points.

Document: {0}

Summary:
""".format(DOC)

response = model.invoke(prompt)

print(response)